<a href="https://colab.research.google.com/github/angelafrocha/Estatistica-com-Python-para-analise-de-dados/blob/main/final_project_veiculos_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#INSTALAÇÃO E IMPORTAÇÃO DE BIBLIOTECAS#

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=8b566ebc121cd31022458c7a6bc6a784b5da0f6eb4b6db8049523d0d16aed9da
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.4.0
    Uninstalling fsspec-2023.4.0:
      Successfully uninstalled fsspec-2023.4.0


In [ ]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 18.6 MB/s eta 0:00:00


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
from pyspark.sql.types import IntegerType, FloatType, DoubleType
import seaborn as sns
import warnings
from google.cloud import storage
import os
import matplotlib.pyplot as plt
import pymongo
from pymongo import MongoClient
warnings.filterwarnings("ignore")

In [ ]:
#montando um drive, para ter acesso a chave do GCP
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#CONFIGURAÇÕES#

In [ ]:
#CONFIGURAR A CHAVE do google cloud
serviceAccount = 'chave_conexão_google'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


In [ ]:
#Acessando a bucket criada
client = storage.Client()

bucket = client.get_bucket('projetofinal1')

bucket.blob('autoveiculos_total.json')



In [ ]:
#Criando uma sessão spark
spark = (SparkSession.builder
                     .master('local')
                     .appName('veiculos')
                     .config('spark.ui.port', '4050')
                     .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                     .getOrCreate())

In [ ]:
#conectando o python ao mongo
uri = "mongodb+srv://clustere5.wjshwzt.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='chave_mongo.pem',
                     )
db = client['prod_venda_join']
intermediario = db['intermediario']

#EXTRAÇÃO#

Extraindo os 6 datasets referentes a produção e vendas de automoveis alocadas na pasta brutos do bucket (datalake)

In [ ]:

df_venda_autoveiculos_total = (
    spark.read
         .format('json')
         .option('delimiter', ',')
         .option('header', 'true')
         .option('inferschema', 'true')
         .option('encoding','utf-8')
         .option('escape','"')
         .load('gs://projetofinal1/brutos/autoveiculos_total.json')
)

In [ ]:
df_venda_comerciais_conces = (
    spark.read
         .format('csv')
         .option('delimiter', ';')
         .option('header', 'true')
         .option('inferschema', 'true')
         .option('encoding','utf-8')
         .option('escape','"')
         .load('gs://projetofinal1/brutos/comerciaisLeves_concessionarias.csv')
)

In [ ]:
df_venda_total_conces = (
    spark.read
         .format('json')
         .option('delimiter', ',')
         .option('header', 'true')
         .option('inferschema', 'true')
         .option('encoding','utf-8')
         .option('escape','"')
         .load('gs://projetofinal1/brutos/total_concessionarias.json')
)

In [ ]:
df_prod_automoveis = (
    spark.read
         .format('csv')
         .option('delimiter', ';')
         .option('header', 'true')
         .option('inferschema', 'true')
         .option('encoding','utf-8')
         .option('escape','"')
         .load('gs://projetofinal1/brutos/prod_automoveis.csv')
)

In [ ]:
df_prod_total_autoveiculos = (
    spark.read
         .format('csv')
         .option('delimiter', ';')
         .option('header', 'true')
         .option('inferschema', 'true')
         .option('encoding','utf-8')
         .option('escape','"')
         .load('gs://projetofinal1/brutos/producao_total_auto.csv')
)

In [ ]:
df_vendas_merc_Interno = (
    spark.read
         .format('csv')
         .option('delimiter', ';')
         .option('header', 'true')
         .option('inferschema', 'true')
         .option('encoding','utf-8')
         .option('escape','"')
         .load('gs://projetofinal1/brutos/vendas_merc_Interno.csv')
)

#Backups#

In [ ]:
df_backup_vendas_inter = df_vendas_merc_Interno

In [ ]:
df__backup_prod_total_autoveiculos = df_prod_total_autoveiculos

In [ ]:
df_backup_prod_automoveis = df_prod_automoveis

In [ ]:
df_backup_venda_total_conces = df_venda_total_conces

In [ ]:
df_backup_venda_comerciais_conces = df_venda_comerciais_conces

In [ ]:
df_backup_venda_autoveiculos_total = df_venda_autoveiculos_total

#ANALISE EXPLORATÓRIA DOS DADOS#

In [ ]:
df_prod_automoveis.show()

+----------+------+
|      data| valor|
+----------+------+
|01/01/1993| 62913|
|01/02/1993| 70663|
|01/03/1993| 93322|
|01/04/1993| 83861|
|01/05/1993| 95587|
|01/06/1993| 96818|
|01/07/1993| 98985|
|01/08/1993|109389|
|01/09/1993|100182|
|01/10/1993|103272|
|01/11/1993|104836|
|01/12/1993| 90390|
|01/01/1994| 88749|
|01/02/1994| 90725|
|01/03/1994|116167|
|01/04/1994| 90052|
|01/05/1994|112323|
|01/06/1994|102368|
|01/07/1994|107274|
|01/08/1994|119641|
+----------+------+
only showing top 20 rows



In [ ]:
df_prod_total_autoveiculos.show()

+----------+------+
|      data| valor|
+----------+------+
|01/01/1993| 66168|
|01/02/1993| 73042|
|01/03/1993| 98691|
|01/04/1993| 88839|
|01/05/1993|101055|
|01/06/1993|102512|
|01/07/1993|104493|
|01/08/1993|115800|
|01/09/1993|106990|
|01/10/1993|109989|
|01/11/1993|111043|
|01/12/1993| 94678|
|01/01/1994| 93357|
|01/02/1994| 95060|
|01/03/1994|122600|
|01/04/1994| 94820|
|01/05/1994|119441|
|01/06/1994|109101|
|01/07/1994|114188|
|01/08/1994|127038|
+----------+------+
only showing top 20 rows



In [ ]:
df_venda_autoveiculos_total.show()

+----------+-----+
|      data|valor|
+----------+-----+
|01/01/1981|87939|
|01/02/1981|65251|
|01/03/1981|64065|
|01/04/1981|69042|
|01/05/1981|62966|
|01/06/1981|61271|
|01/07/1981|60824|
|01/08/1981|63871|
|01/09/1981|64828|
|01/10/1981|63211|
|01/11/1981|61129|
|01/12/1981|68939|
|01/01/1982|62413|
|01/02/1982|65785|
|01/03/1982|74433|
|01/04/1982|67611|
|01/05/1982|64058|
|01/06/1982|69819|
|01/07/1982|72763|
|01/08/1982|75621|
+----------+-----+
only showing top 20 rows



In [ ]:
df_venda_comerciais_conces.show()

+----------+-----+
|      data|valor|
+----------+-----+
|01/01/1990|13195|
|01/02/1990|10715|
|01/03/1990| 5822|
|01/04/1990| 5531|
|01/05/1990| 9734|
|01/06/1990| 7569|
|01/07/1990| 8296|
|01/08/1990|15328|
|01/09/1990|13247|
|01/10/1990|15010|
|01/11/1990|12234|
|01/12/1990|11751|
|01/01/1991| 9631|
|01/02/1991| 9320|
|01/03/1991|10584|
|01/04/1991| 5978|
|01/05/1991| 8612|
|01/06/1991|12006|
|01/07/1991|15026|
|01/08/1991|12955|
+----------+-----+
only showing top 20 rows



In [ ]:
df_vendas_merc_Interno.show()

+----------+-----+
|      data|valor|
+----------+-----+
|01/01/1981|73363|
|01/02/1981|48973|
|01/03/1981|40523|
|01/04/1981|48580|
|01/05/1981|41750|
|01/06/1981|45874|
|01/07/1981|42017|
|01/08/1981|44446|
|01/09/1981|47095|
|01/10/1981|48790|
|01/11/1981|47231|
|01/12/1981|52008|
|01/01/1982|48208|
|01/02/1982|48093|
|01/03/1982|57707|
|01/04/1982|52270|
|01/05/1982|50905|
|01/06/1982|56510|
|01/07/1982|56526|
|01/08/1982|60109|
+----------+-----+
only showing top 20 rows



In [ ]:
df_vendas_merc_Interno.count()

507

In [ ]:
df_vendas_merc_Interno.drop_duplicates().count()

507

In [ ]:
df_venda_total_conces.count()

399

In [ ]:
df_venda_total_conces.drop_duplicates().count()

399

In [ ]:
df_prod_automoveis.count()

363

In [ ]:
df_prod_automoveis.drop_duplicates().count()

363

In [ ]:
df_venda_autoveiculos_total.count()

507

In [ ]:
df_venda_autoveiculos_total.drop_duplicates().count()

507

In [ ]:
df_venda_comerciais_conces.count()

399

In [ ]:
df_venda_comerciais_conces.drop_duplicates().count()

399

In [ ]:
df_venda_autoveiculos_total.show()

+----------+-----+
|      data|valor|
+----------+-----+
|01/01/1981|87939|
|01/02/1981|65251|
|01/03/1981|64065|
|01/04/1981|69042|
|01/05/1981|62966|
|01/06/1981|61271|
|01/07/1981|60824|
|01/08/1981|63871|
|01/09/1981|64828|
|01/10/1981|63211|
|01/11/1981|61129|
|01/12/1981|68939|
|01/01/1982|62413|
|01/02/1982|65785|
|01/03/1982|74433|
|01/04/1982|67611|
|01/05/1982|64058|
|01/06/1982|69819|
|01/07/1982|72763|
|01/08/1982|75621|
+----------+-----+
only showing top 20 rows



In [ ]:
df_vendas_merc_Interno.show()

+----------+-----+
|      data|valor|
+----------+-----+
|01/01/1981|73363|
|01/02/1981|48973|
|01/03/1981|40523|
|01/04/1981|48580|
|01/05/1981|41750|
|01/06/1981|45874|
|01/07/1981|42017|
|01/08/1981|44446|
|01/09/1981|47095|
|01/10/1981|48790|
|01/11/1981|47231|
|01/12/1981|52008|
|01/01/1982|48208|
|01/02/1982|48093|
|01/03/1982|57707|
|01/04/1982|52270|
|01/05/1982|50905|
|01/06/1982|56510|
|01/07/1982|56526|
|01/08/1982|60109|
+----------+-----+
only showing top 20 rows



In [ ]:
df_venda_comerciais_conces.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: integer (nullable = true)



In [ ]:
df_venda_total_conces.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: string (nullable = true)



In [ ]:
df_venda_autoveiculos_total.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: string (nullable = true)



In [ ]:
df_vendas_merc_Interno.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: integer (nullable = true)



In [ ]:
df_prod_automoveis.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: integer (nullable = true)



In [ ]:
df_prod_total_autoveiculos.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: integer (nullable = true)



In [ ]:
df_vendas_merc_Interno.toPandas().isna().any()

data     False
valor    False
dtype: bool

In [ ]:
df_venda_autoveiculos_total.toPandas().isna().any()

data     False
valor    False
dtype: bool

In [ ]:
df_venda_comerciais_conces.toPandas().isna().any()

data     False
valor    False
dtype: bool

In [ ]:
df_prod_automoveis.toPandas().isna().any()

data     False
valor    False
dtype: bool

In [ ]:
df_prod_total_autoveiculos.toPandas().isna().any()

data     False
valor    False
dtype: bool

#Transformação#

In [ ]:
df_prod_total_autoveiculos.withColumn('data', F.to_date('data','dd/MM/yyyy')).show(3)

+----------+-----+
|      data|valor|
+----------+-----+
|1993-01-01|66168|
|1993-02-01|73042|
|1993-03-01|98691|
+----------+-----+
only showing top 3 rows



In [ ]:
#Convertendo o campo data para o formato date
df_prod_total_autoveiculos = df_prod_total_autoveiculos.withColumn('data', F.to_date('data','dd/MM/yyyy'))

In [ ]:
df_prod_total_autoveiculos.show(3)

+----------+-----+
|      data|valor|
+----------+-----+
|1993-01-01|66168|
|1993-02-01|73042|
|1993-03-01|98691|
+----------+-----+
only showing top 3 rows



In [ ]:
df_prod_total_autoveiculos.printSchema()

root
 |-- data: date (nullable = true)
 |-- valor: integer (nullable = true)



In [ ]:
#Renomeando o campo valor para melhor identificação do campo após realizar o join
df_prod_total_autoveiculos = df_prod_total_autoveiculos.withColumnRenamed('valor', 'Producao_Autoveiculos_total')

In [ ]:
df_prod_total_autoveiculos.show(3)

+----------+---------------------------+
|      data|Producao_Autoveiculos_total|
+----------+---------------------------+
|1993-01-01|                      66168|
|1993-02-01|                      73042|
|1993-03-01|                      98691|
+----------+---------------------------+
only showing top 3 rows



Renomeando o campo valor para melhor identificação do campo após realizar o join e convertenda a data

In [ ]:
df_prod_automoveis = df_prod_automoveis.withColumn('data', F.to_date('data','dd/MM/yyyy'))

In [ ]:
df_prod_automoveis = df_prod_automoveis.withColumnRenamed('valor', 'Producao_Automoveis')

In [ ]:
df_prod_automoveis.show(3)

+----------+-------------------+
|      data|Producao_Automoveis|
+----------+-------------------+
|1993-01-01|              62913|
|1993-02-01|              70663|
|1993-03-01|              93322|
+----------+-------------------+
only showing top 3 rows



Renomeando o campo valor para melhor identificação do campo após realizar o join e convertenda a data

In [ ]:
df_venda_autoveiculos_total = df_venda_autoveiculos_total.withColumn('data', F.to_date('data','dd/MM/yyyy'))

In [ ]:
df_venda_autoveiculos_total = df_venda_autoveiculos_total.withColumnRenamed('valor', 'valorVendas_autoveiculosTotal')

In [ ]:
df_venda_autoveiculos_total.show(3)

+----------+-----------------------------+
|      data|valorVendas_autoveiculosTotal|
+----------+-----------------------------+
|1981-01-01|                        87939|
|1981-02-01|                        65251|
|1981-03-01|                        64065|
+----------+-----------------------------+
only showing top 3 rows



In [ ]:
df_venda_comerciais_conces = df_backup_venda_total_conces

In [ ]:
df_venda_comerciais_conces.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: string (nullable = true)



Renomeando o campo valor para melhor identificação do campo após realizar o join e convertenda a data

In [ ]:
df_venda_comerciais_conces = df_venda_comerciais_conces.withColumn('data', F.to_date('data','dd/MM/yyyy'))

In [ ]:
df_venda_comerciais_conces = df_venda_comerciais_conces.withColumnRenamed('valor', 'valorVendas_ConcesComerciais')

In [ ]:
df_venda_comerciais_conces.printSchema()

root
 |-- data: date (nullable = true)
 |-- valorVendas_ConcesComerciais: string (nullable = true)



In [ ]:
df_venda_comerciais_conces.show(3)

+----------+----------------------------+
|      data|valorVendas_ConcesComerciais|
+----------+----------------------------+
|1990-01-01|                       69792|
|1990-02-01|                       57258|
|1990-03-01|                       32740|
+----------+----------------------------+
only showing top 3 rows



Renomeando o campo valor para melhor identificação do campo após realizar o join e convertenda a data

In [ ]:
df_vendas_merc_Interno = df_vendas_merc_Interno.withColumn('data', F.to_date('data','dd/MM/yyyy'))

In [ ]:
df_vendas_merc_Interno = df_vendas_merc_Interno.withColumnRenamed('valor', 'valorVendas_mercadoInterno')

In [ ]:
df_vendas_merc_Interno.show(3)

+----------+--------------------------+
|      data|valorVendas_mercadoInterno|
+----------+--------------------------+
|1981-01-01|                     73363|
|1981-02-01|                     48973|
|1981-03-01|                     40523|
+----------+--------------------------+
only showing top 3 rows



In [ ]:
df_venda_total_conces.show(3)

+----------+-----+
|      data|valor|
+----------+-----+
|01/01/1990|69792|
|01/02/1990|57258|
|01/03/1990|32740|
+----------+-----+
only showing top 3 rows



Renomeando o campo valor para melhor identificação do campo após realizar o join e convertenda a

In [ ]:
df_venda_total_conces = df_venda_total_conces.withColumn('data', F.to_date('data','dd/MM/yyyy'))

In [ ]:
df_venda_total_conces = df_venda_total_conces.withColumnRenamed('valor', 'valorVendas_concesTotal')

In [ ]:
df_venda_total_conces.printSchema()

root
 |-- data: date (nullable = true)
 |-- valorVendas_concesTotal: string (nullable = true)



In [ ]:
df_venda_total_conces.show(10)

+----------+-----------------------+
|      data|valorVendas_concesTotal|
+----------+-----------------------+
|1990-01-01|                  69792|
|1990-02-01|                  57258|
|1990-03-01|                  32740|
|1990-04-01|                  32812|
|1990-05-01|                  58464|
|1990-06-01|                  37632|
|1990-07-01|                  43697|
|1990-08-01|                  84707|
|1990-09-01|                  75195|
|1990-10-01|                  80938|
+----------+-----------------------+
only showing top 10 rows



In [ ]:
df_venda_comerciais_conces.show(10)

+----------+----------------------------+
|      data|valorVendas_ConcesComerciais|
+----------+----------------------------+
|1990-01-01|                       69792|
|1990-02-01|                       57258|
|1990-03-01|                       32740|
|1990-04-01|                       32812|
|1990-05-01|                       58464|
|1990-06-01|                       37632|
|1990-07-01|                       43697|
|1990-08-01|                       84707|
|1990-09-01|                       75195|
|1990-10-01|                       80938|
+----------+----------------------------+
only showing top 10 rows



In [ ]:
df_vendas_merc_Interno.count()

507

In [ ]:
df_venda_total_conces.count()

399

In [ ]:
df_venda_comerciais_conces.count()

399

In [ ]:
df_venda_autoveiculos_total.count()

507

In [ ]:
df_prod_automoveis.count()

363

In [ ]:
df_prod_total_autoveiculos.count()

363

Realizando o join dos dataframes, iniciando o df com maior número de linhas para o menor

In [ ]:
df_join = df_vendas_merc_Interno.join(df_prod_total_autoveiculos, on=['data'], how='left')

In [ ]:
df_join = df_join.join(df_venda_autoveiculos_total, on=['data'], how='left')

In [ ]:
df_join = df_join.join(df_prod_automoveis, on=['data'], how='left')

In [ ]:
df_join = df_join.join(df_venda_total_conces, on=['data'], how='left')

In [ ]:
df_join = df_join.join(df_venda_comerciais_conces, on=['data'], how='left')

In [ ]:
df_join.show(3)

+----------+--------------------------+---------------------------+-----------------------------+-------------------+-----------------------+----------------------------+
|      data|valorVendas_mercadoInterno|Producao_Autoveiculos_total|valorVendas_autoveiculosTotal|Producao_Automoveis|valorVendas_concesTotal|valorVendas_ConcesComerciais|
+----------+--------------------------+---------------------------+-----------------------------+-------------------+-----------------------+----------------------------+
|1981-01-01|                     73363|                       null|                        87939|               null|                   null|                        null|
|1981-02-01|                     48973|                       null|                        65251|               null|                   null|                        null|
|1981-03-01|                     40523|                       null|                        64065|               null|                   null|    

In [ ]:
df_join = df_join.join(df_prod_automoveis, on=['data'], how='left')

In [ ]:
df_join.show(3)

+----------+--------------------------+---------------------------+-----------------------------+-------------------+-----------------------+----------------------------+-------------------+
|      data|valorVendas_mercadoInterno|Producao_Autoveiculos_total|valorVendas_autoveiculosTotal|Producao_Automoveis|valorVendas_concesTotal|valorVendas_ConcesComerciais|Producao_Automoveis|
+----------+--------------------------+---------------------------+-----------------------------+-------------------+-----------------------+----------------------------+-------------------+
|1981-01-01|                     73363|                       null|                        87939|               null|                   null|                        null|               null|
|1981-02-01|                     48973|                       null|                        65251|               null|                   null|                        null|               null|
|1981-03-01|                     40523|      

In [ ]:
df_join.printSchema()

root
 |-- data: date (nullable = true)
 |-- valorVendas_mercadoInterno: integer (nullable = true)
 |-- Producao_Autoveiculos_total: integer (nullable = true)
 |-- valorVendas_autoveiculosTotal: string (nullable = true)
 |-- Producao_Automoveis: integer (nullable = true)
 |-- valorVendas_concesTotal: string (nullable = true)
 |-- valorVendas_ConcesComerciais: string (nullable = true)
 |-- Producao_Automoveis: integer (nullable = true)



In [ ]:
df_join.where(F.col('data') == '2015-01-01').show()

+----------+--------------------------+---------------------------+-----------------------------+-------------------+-----------------------+----------------------------+-------------------+
|      data|valorVendas_mercadoInterno|Producao_Autoveiculos_total|valorVendas_autoveiculosTotal|Producao_Automoveis|valorVendas_concesTotal|valorVendas_ConcesComerciais|Producao_Automoveis|
+----------+--------------------------+---------------------------+-----------------------------+-------------------+-----------------------+----------------------------+-------------------+
|2015-01-01|                    208143|                     204716|                       224445|             194123|                 253788|                      253788|             194123|
+----------+--------------------------+---------------------------+-----------------------------+-------------------+-----------------------+----------------------------+-------------------+



In [ ]:
df_join = df_join.withColumnRenamed('valor', 'valorVendas_autoveiculosTotal')

#CARREGAMENTO#

In [ ]:
#Salvando o DataFrame em um arquivo CSV
df_join.write.format("csv").option("header", "true").mode("overwrite").save("gs://projetofinal1/temp/vendas_prod_veiculos.csv")